In [1]:
# KATHIMERINI

# Import necessary libraries
from bs4 import BeautifulSoup
import requests

# Prompt user to input the tag URL and append 'page/1/' to it for pagination
url = input('Please give m the URL')
url = url+'page/1/'

# Prompt user to input the tag name, used for file naming
tname = input('print the tag name')

# Construct output filenames based on tag name
a = 'Kathimerini'
b = '_Wdate'
csv = '.csv'
simple = a+tname+csv        # CSV file with tag frequencies
wd = a+tname+b+csv          # CSV file linking tags to dates

# Lists to store final tags and corresponding dates
final_tag_list = []
final_date_list = []

# Date threshold (YYYYMMDD); stops scraping when an article older than this is found
searching_until = 20230731
flag = True   # Control variable for while loop

# Function to increment the page number in the URL
def new_url():
    global url
    number = []
    n = 0
    cnt_thesi = 0
    for i in url:
        nr = i.isnumeric()
        if nr == True:
            cnt_thesi += 1
            number.append(i)
            if cnt_thesi == 1:
                thesi = n
        n += 1
    number = "".join(number)     # Convert list to string
    number = int(number) + 1     # Increment page number
    number = str(number)
    url = url[:thesi] + number + "/"   # Replace old number in URL with new one

# Messenger flag indicates whether an article failed to load properly
messanger = False

# Core function to parse a single page of articles
def core_function(page_url, searching_until):
    global final_tag_list
    global final_date_list
    global flag
    global messanger

    inside_tag_list = []  # Tags for one article

    # Function to extract and check the date of an article
    def date_checker(article_url):
        global flag
        global messanger
        page = requests.get(article_url)
        soup = BeautifulSoup(page.content, 'html.parser')
        try:
            # First attempt: look for <time> tag
            time = soup.find('time')
            if time == None:
                # Fallback method: look for alternative date span
                print('Atribute Error occured. \n The art url is ' , article_url)
                time = soup.find('span' , class_ = 'entry-meta-date')
                time = time.get_text()
                time = time[:10]
                time = time.split(".")
                time.reverse()
                time = "".join(time)
                time = int(time)
            else:
                # Extract and clean the date
                time = time.get('datetime')
                time = time[:10].replace('-' , "")
                time = int(time)
            # Stop if article is older than threshold
            if time < searching_until:
                print('the date is reached. Stop the loop!!!')
                flag = False
            else:
                final_date_list.append(time)
        except:
            # Log broken article URLs
            print('broken article. Its url is ' , article_url)

    # Function to collect tags from the article
    def tag_saver(article_url):
        global messanger
        messanger = False
        page = requests.get(article_url)
        soup = BeautifulSoup(page.content, 'html.parser')

        mother = soup.find("div" , class_ = "tags-wrapper")
        if mother == None:
            print('Attention! vrika xalasmeno arthro me url: ' , article_url)
            messanger = True
        else:
            try:
                find_a = mother.find_all('a')
                for a in find_a:
                    tag = a.get_text()
                    inside_tag_list.append(tag)
            except:
                print('PLease check the article url. \n its Propably broken. ' , article_url)

    # Parse current page of articles
    page = requests.get(page_url)
    soup = BeautifulSoup(page.content, 'html.parser')
   
    # Get all articles on the page
    media_content = soup.find_all('div' , class_ = "media-content")

    # If no articles found, stop the loop
    if len(media_content) == 0 :
        print('len(media_content) == 0')
        flag = False

    # Loop over each article block
    for i in media_content:
        article_url = i.find('a')
        article_url = article_url.get('href')

        date_checker(article_url)  # Check date
        if flag == True:
            tag_saver(article_url)  # Save tags

            outer_tag = i.find('p' , class_ = "subtitle is-7 mb-3")
            outer_tag = outer_tag.get_text()

            # Catch mechanism: only save if article was valid
            if messanger == False:
                inside_tag_list.append(outer_tag)
                final_tag_list.append(inside_tag_list)
            else:
                print('the messanger is True. The article is broken and neither tag nor date is appended. See it at the end , comparing their lengths')
            inside_tag_list = []  # Reset for next article
        else:
            break

    # Stop loop if less than 15 articles on page (end of pagination)
    if len(media_content) < 15:
        print('len(media_content) < 15')
        flag = False

# Main loop: continues fetching pages until flag is set to False
flag = True
while flag:
    core_function(url,searching_until)
    if flag == False:
        print('the while loop has stopped immediatly')
        break
    new_url()  # Move to next page

# Print summary stats after scraping
print(len(final_date_list))
print(len(final_tag_list))
print('the last date collected is ' , final_date_list[-1])
print(final_tag_list)

# ---- Data Processing ---- #

import pandas as pd

# Flatten final_tag_list to a single list of tags
new_tag_list = []
for i in range(len(final_tag_list)):
    for j in range(len(final_tag_list[i])):
        new_tag_list.append(final_tag_list[i][j])

# Count occurrences of each tag
res = {}
for i in new_tag_list:
    res[i] = new_tag_list.count(i)

# Convert tag frequency dictionary to list of [tag, count]
resultList = new_list = list(map(list, res.items()))

# Export tag frequencies to CSV
df_costliness = pd.DataFrame(resultList)
df_costliness.to_csv(simple, header=False)

# Format dates into DD.MM.YYYY
ddate_list = final_date_list
ttag_list = final_tag_list

ndlist = []
for i in ddate_list:
    i = str(i)
    y = i[:4]
    m = i[4:6]
    d = i[6:]
    ddate = d+'.'+m+'.'+y
    ndlist.append(ddate)

# Link each tag to its corresponding date
n = 0
date_con_list = []
for i in ttag_list:
    n = n + 1
    for tag in i:
        inside = []
        inside.append(tag)
        inside.append(ddate_list[n-1])
        date_con_list.append(inside)

# Export tag-date pairs to CSV
df_connected = pd.DataFrame(date_con_list)
df_connected.to_csv(wd, header=False)

# Final output summaries
print(len(final_tag_list))
print(len(final_date_list))
print(final_tag_list)
print('the last date collected is ' , final_date_list[-1])
print('the nr of the tags mentioned is ' , len(resultList))


Atribute Error occured. 
 The art url is  https://www.kathimerini.gr/visual/infographics/563639377/stathero-to-eyry-dimoskopiko-provadisma-n-d/
Atribute Error occured. 
 The art url is  https://www.kathimerini.gr/visual/infographics/563637232/anataraxeis-sta-kommata-apo-to-mparaz-ton-dimoskopiseon/
Atribute Error occured. 
 The art url is  https://www.kathimerini.gr/visual/infographics/563617360/o-chartis-ton-diarroon/
Atribute Error occured. 
 The art url is  https://www.kathimerini.gr/visual/infographics/563587330/dimoskopisi-apodochi-metron-stirixis-kai-kerdi-gia-ti-n-d/
Atribute Error occured. 
 The art url is  https://www.kathimerini.gr/visual/infographics/563525173/dimoskopiki-aimorragia/
Atribute Error occured. 
 The art url is  https://www.kathimerini.gr/visual/infographics/563489134/ta-simadia-toy-megaloy-thymoy/
Atribute Error occured. 
 The art url is  https://www.kathimerini.gr/visual/infographics/563481337/sto-15-oi-anapofasistoi/
Atribute Error occured. 
 The art url is  